#  Table of Contents

1. [Introduction](#1-introduction)
    * 1.1 What the notebook covers
    * 1.2 Why this topic/project matters
2. [Background](#2-background)
    * 2.1 Brief on graphs and semantic networks
    * 2.2 Contextual link to Jakobson’s model and emotional triggers (optional but powerful for narrative)
    * 2.3 Any relevant problem framing (e.g., NLP or emotion modeling motivation)
3. [Objectives](#3-objectives)
    * 3.1 What you aim to demonstrate
    * 3.2 Expected outcomes of the analysis
4. [Setup](#4-setup)
    * 4.1 Install/import libraries
    * 4.2 Load and preview data
5. [Preprocessing](#5-preprocessing)
    * 5.1 Data cleaning, NLP (concept extraction, emotion tagging)
    * 5.2 Construction of nodes and edges
    * 5.3 Create initial DataFrames or dictionaries
6. [Graph Construction](#6-graph-construction)
    * 6.1 Building the graph using NetworkX
    * 6.2 Directed vs undirected, weights, labels, etc.
    * 6.3 Visualizations (basic layout, color by emotion or function)
7. [Graph Analysis](#7-graph-analysis)
    * 7.1 Centrality, clustering, shortest path, connected components
    * 7.2 Subgraph exploration (e.g., emotion-specific clusters)
8. [Advanced Visualization](#8-advanced-visualization)
    * 8.1 Using pyvis, Plotly, or improved NetworkX visualizations
    * 8.2 Interactive graph options if desired
9. [Insights & Interpretation](#9-insights--interpretation)
    * 9.1 What the graph tells you about emotional triggers, semantic structure, or Jakobsonian functions
    * 9.2 Highlight surprising or significant findings
10. [Conclusion](#10-conclusion)
    * 10.1 Summary of learnings
    * 10.2 Next steps or extensions
11. [Appendix (Optional)](#11-appendix-optional)
    * 11.1 Notes on concept embeddings, model details, references
    * 11.2 Runtime notes, parameter tweaks, etc.

### 1. Introduction

        1.1 What the notebook covers


        1.2 Why this topic/project matters







### 2. Background


        2.1 Brief on graphs and semantic networks
    
    
    
        2.2 Contextual link to Jakobson’s model and emotional triggers (optional but powerful for narrative)
    
    
    
    
        2.3 Any relevant problem framing (e.g., NLP or emotion modeling motivation)

### 3. Objectives


        3.1 What you aim to demonstrate


        3.2 Expected outcomes of the analysis

### 4. Setup

        4.1 Install/import libraries

In [1]:
%pip install -e ..

Obtaining file:///C:/Users/britt/source/repos/notebook-to-prod-template
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for notebook-to-prod-template (pyproject.toml): started
  Building editable for notebook-to-prod-template (pyproject.toml): finished with status 'done'
  Created wheel for notebook-to-prod-template: filename=notebook_to_prod_template-0.2.0.post4-0.editable-py3-none-any.whl size=6061 sha256=9710e4d349931c6253a3a529f7d78d9905e1e616f43676b4024fe469cbbd8f3f
  Stored in directory: C:\Users\britt\AppData\Local\Temp

In [2]:
from notebook_service.cli import analyze_df
import pandas as pd

        4.2 Load and preview data

In [3]:
df = pd.read_csv("../data/input.csv")
nlu_df = analyze_df(df, text_column="entries")
nlu_df.head()

,entries,sadness,joy,fear,disgust,anger,concepts,semantic_roles
0,"I felt anxious today, so I impulsively ordered...",0.332842,0.049939,0.104065,0.005845,0.212283,"[Delivery (commerce), Anxiety, Restaurant, Foo...","[(I, felt, anxious), (I, ordered, food delivery)]"
1,"After my promotion, I confidently adjusted my ...",0.074566,0.334086,0.117670,0.033797,0.014482,"[Retirement, Fundraising, Ageing, Old age, Soc...","[(I, adjusted, my retirement contributions)]"
2,Stress led me to overspend on shopping yesterday.,0.665364,0.122717,0.186756,0.003421,0.029566,"[Light-emitting diode, Retail, Comparison shop...","[(Stress, led, me)]"
3,Today was incredibly frustrating; I spent hour...,0.246144,0.081976,0.028637,0.018688,0.129494,"[Breakpoint, Debugger, In-circuit emulation, T...","[(Today, was, frustrating; I spent hours debug..."
4,Our sprint demo went perfectly—I’m feeling pro...,0.482779,0.598429,0.011331,0.011596,0.010271,"[Sprint, Sprint (cycling), Flat racing, Sprint...","[(Our sprint demo, went, ), (I, ’m feeling, pr..."


### 5. Preprocessing

        5.1 Data cleaning, NLP (concept extraction, emotion tagging)

In [4]:
concepts = nlu_df["concepts"]
pd.set_option("display.max_colwidth", None)
concepts

0                                                                                        [Delivery (commerce), Anxiety, Restaurant, Food storage, Homburg (hat), Fedora, Valenki, Bowler hat]
1                                                  [Retirement, Fundraising, Ageing, Old age, Social Security (United States), Mandatory retirement, Pension, Social Security Administration]
2                  [Light-emitting diode, Retail, Comparison shopping website, Convenience store, Economics of Christmas, Department store, Central business district, Periodical literature]
3                                                               [Breakpoint, Debugger, In-circuit emulation, Tracing (software), Assembly language, Software bug, Software testing, Software]
4                                                        [Sprint, Sprint (cycling), Flat racing, Sprint car racing, Sprint Cup Series, Sprint Corporation, Eastern Sprints, Sprint (missile)]
5                                                 

        5.2 Construction of nodes and edges

        5.3 Create initial DataFrames or dictionaries